<a href="https://colab.research.google.com/github/astrovishalthakur/MachineLearning/blob/main/FeatureEngineering/EncodingNumericalData/Binarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd


In [24]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import Binarizer

In [10]:
df = pd.read_csv("https://raw.githubusercontent.com/astrovishalthakur/100-days-of-machine-learning/main/day32-binning-and-binarization/train.csv")[["Age", "Fare", "Parch", "SibSp", "Survived"]]

In [11]:
df.dropna(inplace=True)

In [12]:
df.head()

,Age,Fare,Parch,SibSp,Survived
0,22.0,7.2500,0,1,0
1,38.0,71.2833,0,1,1
2,26.0,7.9250,0,0,1
3,35.0,53.1000,0,1,1
4,35.0,8.0500,0,0,0


In [13]:
df["family"] = df.SibSp + df.Parch

In [15]:
df.head()

,Age,Fare,Parch,SibSp,Survived,family
0,22.0,7.2500,0,1,0,1
1,38.0,71.2833,0,1,1,1
2,26.0,7.9250,0,0,1,0
3,35.0,53.1000,0,1,1,1
4,35.0,8.0500,0,0,0,0


In [16]:
df.drop(columns=["Parch", "SibSp"], inplace=True)

In [17]:
df.head()

,Age,Fare,Survived,family
0,22.0,7.2500,0,1
1,38.0,71.2833,1,1
2,26.0,7.9250,1,0
3,35.0,53.1000,1,1
4,35.0,8.0500,0,0


In [18]:
X = df.drop(columns=["Survived"])
y = df["Survived"]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
X_train.head()

,Age,Fare,family
328,31.0,20.5250,2
73,26.0,14.4542,1
253,30.0,16.1000,1
719,33.0,7.7750,0
666,25.0,13.0000,0


In [22]:
# without Binarization
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy_score(y_pred, y_test)

0.6223776223776224

In [23]:
np.mean(cross_val_score(clf, X, y, scoring="accuracy"))

0.6540628385698808

In [25]:
# Applying Binarization
# we can check if a person is travelling alone based on family column

trf = ColumnTransformer(
    [
     ("bin", Binarizer(copy=False), ["family"])
    ], remainder="passthrough"
)

In [26]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [28]:
X_train_trf[:5]

array([[ 1.    , 31.    , 20.525 ],
       [ 1.    , 26.    , 14.4542],
       [ 1.    , 30.    , 16.1   ],
       [ 0.    , 33.    ,  7.775 ],
       [ 0.    , 25.    , 13.    ]])

In [27]:
pd.DataFrame(X_train_trf, columns=["family", "Age", "Fare"])

,family,Age,Fare
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


In [29]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf, y_train)

y_pred2 = clf.predict(X_test_trf)

accuracy_score(y_test, y_pred2)

0.6153846153846154

In [30]:
X_trf = trf.fit_transform(X)

np.mean(cross_val_score(DecisionTreeClassifier(), X_trf, y, cv=10, scoring="accuracy"))

0.6318270735524256